<a href="https://colab.research.google.com/github/ChiaoYunTing/ADA-Group15/blob/main/neural_loan_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:

from google.colab import files

uploaded = files.upload()

In [ ]:
# upload an in-built Python (OK semi-in-built) dataset
import pandas as pd
import numpy as np

df = pd.read_excel('loan_data.xlsx')

In [ ]:
target = df.pop('loan_is_bad')
features = df
tf_dataset = tf.data.Dataset.from_tensor_slices((features.values, target.values))

In [ ]:
n = len(list(tf_dataset))
train_size = int(n * 0.8)
train_dataset = tf_dataset.take(train_size)
test_dataset = tf_dataset.skip(train_size)

In [ ]:
train_batch = train_dataset.batch(50) # batch size = 50
features, labels = next(iter(train_batch)) # iterate through each batch at training time

# print the first 20 records (rows) of features and labels
print(features[0:20])
print(labels[0:20])

In [ ]:

feature_count = features.shape[1]#determine number of features
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(feature_count,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])#dropout rate could be 0.2-0.5 and it is for reducing overfitting

In [ ]:
# compile the model
model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001,  weight_decay=0.005),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])#learning_rate and weight_decay are for reducing overfitting(idk whether it is overlapped with dropout rate)
model.summary()

In [ ]:
history = model.fit(train_dataset, epochs=10)

In [ ]:

test_dataset = test_dataset.batch(50)
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=2)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:

predictions = model.predict(test_dataset)
for pred, real in zip(predictions, test_labels):
    predicted_label = (pred > 0.5).astype(int)
    print(f"Predicted: {predicted_label};    Real: {real}")
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=2)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")